# Model Trainer
<img src="https://images.ctfassets.net/wt70guc1rpin/wp-media-31995/b7c03abc540764a4735247d34b590d49/aW8ehS-tyXOhnQL5U103N77RXn4R5QwNGgoq2-Ba3L1NN45gbXq0tX7QQAlUf5LCzehQxVoVOzRoKZL5uK3PsT-rmIRfu-aPDree5fgHGJjb0Fpb6TehBv8Zg9hx.png">

In [1]:
# to subpress warning
import warnings
warnings.filterwarnings("ignore")


# The OS module in Python provides functions for interacting with the operating system.
import os

# Matplotlib is a data visualization and graphical plotting library for Python.
import matplotlib.pyplot as plt

# seaborn is alse a data visualization and graphical plotting library for Python.
import seaborn as sn

# Used to display markdown,image,control (frontend utilities)
from IPython.display import display, clear_output

# NumPy is a Python library used for working with arrays
import numpy as np

import pandas as pd

import pickle

# used to split dataset(features and target) into test and test
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix,classification_report

# shuffle the dataset for a even mixture of each type of feature and target.it gives better result.
from sklearn.utils import shuffle

# Computer vision library
import cv2

# Used to manipulate different parts of the Python runtime environment.
import sys

# An open-source framework for building pipelines to perform computer vision inference.
import mediapipe as mp

# Time module provide time-related functions
import time

# Used to generate random numbers
import random

from math import log10, sqrt
# for mathematical operations

# Keras is a library that provides a Python interface for artificial neural networks. 
# Keras acts as an interface for the TensorFlow library.
import keras

# 1. Keras layers are the building blocks of the Keras library that can be stacked together for creating neural network models.
# 2. Keras Conv2D creates a 2D convolution kernel that is wind with layers input which helps produce a tensor of outputs.
# 3. maxpooling2D Downsamples the input along its spatial dimensions by taking the maximum value over an input window for each channel of the input. 
# 4 .Flattening is converting the data into a 1-dimensional array for inputting it to the next layer. 
# 5 .Dropout regularization technique for reducing overfitting in neural networks by preventing complex co-adaptations on training data.
# 6. Batch normalization is a technique for training very deep neural networks that standardizes the inputs to a layer for each mini-batch. 
from tensorflow.keras.layers import Conv2D, MaxPool2D, Flatten, Dense, Dropout, BatchNormalization

#  Model groups layers into an object with training and inference features.
from tensorflow.keras.models import Model

import tensorflow as tf

# Optimizers are the extended class, which include added information to train a specific model.  
# The optimizers are used for improving speed and performance for training a specific model.
# Adam is a stochastic gradient descent method based on adaptive estimation of first-order and second-order moments.
from tensorflow.keras.optimizers import Adam

# importing Sqquential model
from tensorflow.keras import Sequential

# tensorflow.keras.callbacks is used to visualize training of a model.
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint 

In [2]:
# GPU Info 
try:
    import GPUtil
except:
    !pip install GPUtil
    import GPUtil

    
# check physical computing devices
device=tf.config.experimental.list_physical_devices()
for i in device:
    print(i)
if len(device)>1:
    # find GPU details
    print("="*20, "GPU Details", "="*20)
    gpus = GPUtil.getGPUs()
    for gpu in gpus:
        print(gpu_id,gpu.name, gpu.driver,gpu.memoryTotal,gpu.temperature)



PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')


In [3]:
PATH=r"D:/Image_datasets/asl_dataset"

In [4]:
# Loading data from CSV and preprocessing
def load_CSV(dataset="dataset.csv"):
    import pandas as pd
    df = pd.read_csv(dataset,header=None,index_col=None)
    
    return df
df1=load_CSV()
df1.head()

,0,1,2,3,4,5,6,7,8,9,...,35,36,37,38,39,40,41,42,43,44
0,0/hand1_0_bot_seg_1_cropped.jpeg,Right,123,133,136,130,154,117,179,111,...,66,155,24,216,34,248,51,268,64,0
1,0/hand1_0_bot_seg_2_cropped.jpeg,Right,129,107,141,113,164,100,201,92,...,56,155,6,206,13,234,31,253,45,0
2,0/hand1_0_bot_seg_3_cropped.jpeg,Right,125,102,141,92,158,76,189,70,...,69,138,30,186,28,215,42,236,54,0
3,0/hand1_0_bot_seg_4_cropped.jpeg,Right,141,122,150,133,157,136,164,145,...,91,206,12,258,32,280,58,293,78,0
4,0/hand1_0_bot_seg_5_cropped.jpeg,Left,135,152,153,148,174,123,196,101,...,80,160,32,219,45,247,65,265,81,0


In [5]:
unique_hand=df1[1].unique()
unique_sign=df1[44].unique()
print(unique_hand)
print(unique_sign)
print(df1[1].value_counts())
print(df1[44].value_counts())

['Right' 'Left']
['0' '1' '2' '3' '4' '5' '6' '7' '8' '9' 'a' 'b' 'c' 'd' 'e' 'f' 'g' 'h'
 'i' 'j' 'k' 'l' 'm' 'n' 'o' 'p' 'q' 'r' 's' 't' 'u' 'v' 'w' 'x' 'y' 'z']
Left     2448
Right      67
Name: 1, dtype: int64
e    70
0    70
w    70
y    70
a    70
9    70
6    70
b    70
3    70
q    70
u    70
z    70
o    70
j    70
2    70
5    70
7    70
g    70
s    70
4    70
x    70
m    70
c    70
v    70
f    70
r    70
k    70
1    70
n    70
p    70
h    70
l    70
d    70
i    70
8    70
t    65
Name: 44, dtype: int64


In [6]:
df = pd.get_dummies(df1, columns = [1, 44])
df = df.sample(frac = 1)
df.columns

Index([        0,         2,         3,         4,         5,         6,
               7,         8,         9,        10,        11,        12,
              13,        14,        15,        16,        17,        18,
              19,        20,        21,        22,        23,        24,
              25,        26,        27,        28,        29,        30,
              31,        32,        33,        34,        35,        36,
              37,        38,        39,        40,        41,        42,
              43,  '1_Left', '1_Right',    '44_0',    '44_1',    '44_2',
          '44_3',    '44_4',    '44_5',    '44_6',    '44_7',    '44_8',
          '44_9',    '44_a',    '44_b',    '44_c',    '44_d',    '44_e',
          '44_f',    '44_g',    '44_h',    '44_i',    '44_j',    '44_k',
          '44_l',    '44_m',    '44_n',    '44_o',    '44_p',    '44_q',
          '44_r',    '44_s',    '44_t',    '44_u',    '44_v',    '44_w',
          '44_x',    '44_y',    '44_z'],
      dtyp

In [7]:
filenames=df[0]
filenames

1694    o/hand1_o_left_seg_5_cropped.jpeg
2270    w/hand2_w_left_seg_1_cropped.jpeg
1710     o/hand2_o_dif_seg_1_cropped.jpeg
2310     x/hand1_x_dif_seg_1_cropped.jpeg
1046     e/hand5_e_dif_seg_2_cropped.jpeg
                      ...                
2042    t/hand1_t_left_seg_3_cropped.jpeg
375      5/hand2_5_bot_seg_1_cropped.jpeg
1000     e/hand1_e_top_seg_1_cropped.jpeg
2342    x/hand2_x_left_seg_3_cropped.jpeg
892      c/hand3_c_dif_seg_3_cropped.jpeg
Name: 0, Length: 2515, dtype: object

In [8]:
cols=list(range(1,45))
features=df[df.columns[cols]]
features

,2,3,4,5,6,7,8,9,10,11,...,36,37,38,39,40,41,42,43,1_Left,1_Right
1694,151,303,216,269,257,203,293,161,304,129,...,126,153,185,83,247,87,286,109,1,0
2270,203,365,237,317,237,244,184,203,134,188,...,144,217,134,177,150,201,160,232,1,0
1710,136,100,163,81,196,55,223,46,247,44,...,188,53,234,66,260,75,279,82,1,0
2310,207,315,252,285,278,210,232,154,181,129,...,130,189,101,171,120,219,143,236,1,0
1046,190,296,261,240,301,162,258,108,200,135,...,103,144,94,75,128,95,151,132,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2042,162,318,236,276,291,190,264,101,235,47,...,104,152,93,120,107,175,125,201,1,0
375,178,384,243,354,292,292,317,229,346,185,...,112,223,82,180,61,149,45,118,1,0
1000,169,203,211,195,247,173,235,173,191,185,...,86,112,83,90,101,129,116,161,1,0
2342,203,350,234,294,238,216,211,159,183,124,...,146,192,107,168,113,201,136,226,1,0


In [9]:
cols=["44_"+x for x in unique_sign]
targets=df[cols]
targets

,44_0,44_1,44_2,44_3,44_4,44_5,44_6,44_7,44_8,44_9,...,44_q,44_r,44_s,44_t,44_u,44_v,44_w,44_x,44_y,44_z
1694,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2270,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
1710,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2310,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1046,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2042,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
375,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1000,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2342,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [10]:
# the data for training and the remaining 10% for testing
train_x,test_x,train_y,test_y,trainFilenames,testFilenames = train_test_split(features, targets, filenames, test_size=0.10,random_state=42)


In [11]:
print(len(train_x),len(test_x))
print(len(train_y), len(test_y))
print(len(trainFilenames), len(testFilenames))
print(train_x.shape)
print(train_y.shape)

2263 252
2263 252
2263 252
(2263, 44)
(2263, 36)


In [17]:
# KPD23 - Key Point Detection
def kpd23():
    # Create model here
    model = Sequential()
    model.add(Dense(40, input_dim = 44, activation = 'relu')) # Rectified Linear Unit Activation Function
    model.add(Dense(40, activation = 'relu'))
    model.add(Dense(36, activation = 'softmax')) # Softmax for multi-class classification
    # Compile model here
    model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
    
    return model

    
    

In [18]:
model=kpd23()
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 40)                1800      
                                                                 
 dense_4 (Dense)             (None, 40)                1640      
                                                                 
 dense_5 (Dense)             (None, 36)                1476      
                                                                 
Total params: 4,916
Trainable params: 4,916
Non-trainable params: 0
_________________________________________________________________


In [20]:
EPOCH=60
BATCH_SIZE=10
start_time=time.time()

history = model.fit(train_x, train_y, epochs=EPOCH, batch_size = BATCH_SIZE)
training_time=time.time()-start_time

Epoch 1/60
227/227 [==============================] - 1s 6ms/step - loss: 1.1749 - accuracy: 0.7680
Epoch 2/60
227/227 [==============================] - 1s 6ms/step - loss: 0.6349 - accuracy: 0.8418
Epoch 3/60
227/227 [==============================] - 1s 6ms/step - loss: 0.4037 - accuracy: 0.8727
Epoch 4/60
227/227 [==============================] - 1s 6ms/step - loss: 0.4601 - accuracy: 0.8674
Epoch 5/60
227/227 [==============================] - 1s 6ms/step - loss: 0.5576 - accuracy: 0.8498
Epoch 6/60
227/227 [==============================] - 1s 6ms/step - loss: 0.3836 - accuracy: 0.8789
Epoch 7/60
227/227 [==============================] - 1s 6ms/step - loss: 0.3266 - accuracy: 0.8926
Epoch 8/60
227/227 [==============================] - 1s 6ms/step - loss: 0.2684 - accuracy: 0.9081
Epoch 9/60
227/227 [==============================] - 1s 6ms/step - loss: 0.2891 - accuracy: 0.8966
Epoch 10/60
227/227 [==============================] - 1s 6ms/step - loss: 0.3115 - accuracy: 0.9010

In [21]:
model.save("./model.h5", save_format="h5")
print("Training Time : {}".format(round(training_time,2)))

In [22]:
test_object={
    "test_x":test_x,
    "test_y":test_y,
    "testFilenames":testFilenames,
    "unique_sign":unique_sign
}

with open('test_data.pkl', 'wb') as f:
        pickle.dump(test_object, f)